# Sentiment analysis

##  Set up

In [2]:
#%pip install numpy pandas nltk sklearn

In [1]:
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

## Load dataset

In [2]:
df = pd.read_csv(r'../resources/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


16123
12853

## Tiền xử lý dữ liệu

### Làm sạch dữ liệu 
- Loại bỏ các thẻ HTML
- Loại bỏ khoảng trắng thừa và dấu câu 
- Chuyển đổi chữ hoa thành chữ thường 

#### Loại bỏ thẻ HTML

In [3]:
df['review'] = df['review'].str.replace("<br />", "")
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

#### Loại bỏ khoảng trắng thừa và dấu câu

In [4]:
# Hàm để chuyển các ký tự đặc biệt thành khoảng trắng và loại bỏ khoảng trắng thừa
def remove_punctuation(text):
    # Chuyển các ký tự đặc biệt thành khoảng trắng
    text = re.sub(r'[^\w\s]', ' ', text)
    # Loại bỏ các khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [5]:
df['review'] = df['review'].apply(remove_punctuation)

In [6]:
df['review'][16123]

'I am writing this review having watched it several months ago the trailer looked promising enough for me to buy this lame excuse for a movie It is a complete joke and literally a spit in the face of real classics of the early generation of horror like Texas Chainsaw Massacre 1974 which they even had the gall to compare itself to on the back of the cover art The producer who played Brandon should go flip burgers and serve up greasy hamburgers hell he might not even be good at that either The lighting was bad bad bad and a big annoyance through out the film you couldn t even see the actor s faces sometimes I don t even remember the rest of the cast members which is sad really bad they never do anything to impress you to make them memorable That s all the time I will waste on this review PLEASE stay as far away as you can from this pile of junk even if you get it for 25 cents don t do it buy s piece of gum at least IT would keep you entertained If you want good quality low budget fun far

#### Loại bỏ stop words

In [7]:
# Tải stop words từ nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PAVT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
stop_words.remove('no')
stop_words.remove('not')

In [9]:
def remove_stop_words(text):
    words = text.split()
    filter_words = [word for word in words if word not in stop_words]
    return ' '.join(filter_words)

In [13]:
# df['review'] = df['review'].apply(remove_stop_words)
# df['review'][2]

#### Chuyển chữ hoa thành chữ thường 

In [10]:
df['review'] = df['review'].str.lower()
df['review'][16123]

'i am writing this review having watched it several months ago the trailer looked promising enough for me to buy this lame excuse for a movie it is a complete joke and literally a spit in the face of real classics of the early generation of horror like texas chainsaw massacre 1974 which they even had the gall to compare itself to on the back of the cover art the producer who played brandon should go flip burgers and serve up greasy hamburgers hell he might not even be good at that either the lighting was bad bad bad and a big annoyance through out the film you couldn t even see the actor s faces sometimes i don t even remember the rest of the cast members which is sad really bad they never do anything to impress you to make them memorable that s all the time i will waste on this review please stay as far away as you can from this pile of junk even if you get it for 25 cents don t do it buy s piece of gum at least it would keep you entertained if you want good quality low budget fun far

## Chia dữ liệu

In [11]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
from sklearn.model_selection import train_test_split
df_removed = df['review'].apply(remove_stop_words)
X_train, X_test, y_train, y_test = train_test_split(df_removed, df['sentiment'], test_size=0.3, random_state=42)

## Word embeddings

In [12]:
from gensim.models import Word2Vec

In [13]:
sentences = []
for sentence in X_train:
    sentences.append(sentence.split())

In [14]:
model = Word2Vec(sentences, vector_size=200, window=5, sg=1, min_count=5, workers = 80)

In [38]:
model.wv.save('../resources/Word2Vec.txt')

In [43]:
import pickle

# Lưu các vector từ vào tệp nhị phân
with open('../resources/word2vec_vectors.pkl', 'wb') as f:
    pickle.dump(model.wv, f)

In [44]:
import pickle

# Tải lại các vector từ từ tệp nhị phân
with open('../resources/word2vec_vectors.pkl', 'rb') as f:
    word_vectors = pickle.load(f)

# Kiểm tra vector từ đã tải
word_vectors['uninteresting'] - model.wv['uninteresting']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

#### sentence embeddings

In [15]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [16]:
def sum_weights(vectors, w):
    res = np.zeros(len(vectors[0]))
    for i in range(len(vectors)):
        res += w[i]*vectors[i]
    return res

Các tiêu chí đánh giá phim: 
- Kịch bản: 
    + coherent/ incoherent
    + unpredictable/ predictable
- Ý nghĩa phim: meaningful/ meaningless
- Hiệu ứng: impressive / unimpressive
- Cảnh quay: heartfelt / insincere

In [17]:
# list_positive_words = ['heartfelt', 'impress', 'excel', 'meaningful', 'cinematograph']
# list_negative_words = ['dumbsh', 'incoherent', 'dull', 'meaningless', 'ruin']
# list_positive_words = ['heartfelt', 'impress', 'excel', 'meaningful', 'cinematograph']
# list_negative_words = ['dumbsh', 'incoher', 'dull', 'meaningless', 'ruin']

list_positive_words = ['heartfelt', 'gripping', 'impressive', 'meaningful', 'coherent']
list_negative_words = ['insincere', 'predictable', 'soporific', 'illogical', 'uninteresting']

# incoherent == uninteresting
# meaningless == illogical
# unimpressive == soporific
# unpredictable == gripping


In [18]:
for word in list_positive_words:
    print(model.wv.most_similar(word))

print()
for word in list_negative_words:
    print(model.wv.most_similar(word))

[('bittersweet', 0.7970017194747925), ('uplifting', 0.7798797488212585), ('heartwarming', 0.7666594386100769), ('heartbreaking', 0.7534283399581909), ('resonant', 0.7458375096321106), ('sincerity', 0.7437487244606018), ('poignant', 0.7396484613418579), ('affecting', 0.7367772459983826), ('poignancy', 0.7363629937171936), ('harmonious', 0.7347285151481628)]
[('engrossing', 0.7622150182723999), ('taut', 0.755029559135437), ('enthralling', 0.7413274049758911), ('riveting', 0.7403537034988403), ('thrilling', 0.738673210144043), ('twisty', 0.7378712296485901), ('exhilarating', 0.7101288437843323), ('absorbing', 0.7079458832740784), ('rewarding', 0.6953647136688232), ('suspenseful', 0.6922111511230469)]
[('astounding', 0.6560491323471069), ('spectacular', 0.6513088345527649), ('breathtaking', 0.6446487903594971), ('amazing', 0.6272587180137634), ('astonishing', 0.6237916350364685), ('awsome', 0.6232047080993652), ('exhilarating', 0.6227367520332336), ('arresting', 0.6204824447631836), ('prof

In [92]:
# Chưa cần dùng tới
def find_most_dissimilar_words(model, word, top_n=10):
    if word not in model:
        raise ValueError(f"Word '{word}' not in the model vocabulary.")
    
    # Lấy vector của từ cần tìm
    word_vector = model[word]
    
    # Tính khoảng cách cosine giữa từ cần tìm và tất cả các từ khác trong mô hình
    distances = []
    for other_word in model.index_to_key:
        if other_word != word:
            other_word_vector = model[other_word]
            cosine_distance = np.dot(word_vector, other_word_vector) / (np.linalg.norm(word_vector) * np.linalg.norm(other_word_vector))
            distances.append((other_word, cosine_distance))
    
    # Sắp xếp các từ theo khoảng cách cosine tăng dần
    distances.sort(key=lambda x: x[1])
    
    # Lấy top_n từ có khoảng cách cosine lớn nhất (nghĩa khác xa nhất)
    most_dissimilar_words = distances[:top_n]
    
    return most_dissimilar_words

- extraordinary/vivid >< dumbsh

In [19]:
list_positive = [model.wv[word] for word in list_positive_words ]
list_negative = [model.wv[word] for word in list_negative_words ]

In [20]:
for i in range(len(list_positive)):
    print(cosine_similarity(list_positive[i], list_negative[i]))

0.5921916
0.43681884
0.5221902
0.44425625
0.57299125


In [21]:
len(list_positive[0])

200

In [22]:
positive_vector = sum_weights(list_positive, [1,2,1,2,1])
negative_vector = sum_weights(list_negative, [1,2,1,2,1])

In [23]:
cosine_similarity(positive_vector, negative_vector)

0.6333175880350418

In [24]:
def sentence_to_vector(sentence, model):
    word_vectors = [model.wv[word] for word in sentence if word in model.wv] # vector 100 từ
    #dedicates= []
    pos_weights = []
    neg_weights = []
    res = np.zeros(model.vector_size)
    weights = []
    if len(word_vectors) == 0:
        return res
    for i in range(len(word_vectors)):
        word_vector = word_vectors[i]
        checkPositive = cosine_similarity(word_vector, positive_vector) # < 1
        checkNegative = cosine_similarity(word_vector, negative_vector) # < 1
        # if(checkPositive > 1.2*checkNegative):
        #     weights.append(checkPositive)
        # elif checkNegative > 1.2*checkPositive:
        #     weights.append(checkNegative)
        # else: 
        #     weights.append(0.0)
        pos_weights.append(checkPositive)
        neg_weights.append(checkNegative)

    pos_res = sum_weights(word_vectors, pos_weights)
    neg_res = sum_weights(word_vectors, neg_weights)
    return np.concatenate([pos_res, neg_res])

    #return sum_weights(word_vectors, weights)

#### Tạo tập train và test cho model

In [25]:
X_w2v_train = [ sentence_to_vector(sent.split(), model)  for sent in X_train]
X_w2v_test =  [ sentence_to_vector(sent.split(), model)  for sent in X_test]

## Mô hình
chỉ chạy để kiểm thử model Word2Vec

### Hàm khảo sát

In [26]:
from sklearn.metrics import accuracy_score, classification_report
def train_and_valid(model, X_train, y_train, X_test, y_test):
    model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train) 
    y_pred = model.predict(X_test)
    # Đánh giá mô hình
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred)
    print(f"training accuracy: {accuracy_train} \nvalidation accuracy: {accuracy_test}")
    

### Decision tree

In [27]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=42, max_depth=7)
train_and_valid(dt_clf, X_w2v_train,y_train, X_w2v_test, y_test)

training accuracy: 0.8060285714285714 
validation accuracy: 0.771


In [31]:
y_pred = dt_clf.predict(X_w2v_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.76      0.78      0.77      7411
    positive       0.78      0.76      0.77      7589

    accuracy                           0.77     15000
   macro avg       0.77      0.77      0.77     15000
weighted avg       0.77      0.77      0.77     15000



### Logistic regression


In [37]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=42, max_iter=2000)
train_and_valid(log_reg, X_w2v_train, y_train, X_w2v_test, y_test)

training accuracy: 0.8966571428571428 
validation accuracy: 0.8892666666666666


In [36]:
log_reg.n_iter_

array([1795])

### Bảng tổng kết logistic regression với model Word2Vec
Dataset: 
- Loại bỏ các thẻ HTML
- Loại bỏ khoảng trắng thừa và dấu câu 
- Chuyển đổi chữ hoa thành chữ thường 
- Loại bỏ stopwords 

| Tham số Word2Vec | Tham số logistic regression | training accuracy | testing accuracy | 
|----------------|---------------|---------|---------|
|vector_size = 100, window = 5, min_count = 5 | max_iter = 600 | 0.878 | 0.876 |
|vector_size = 200, window = 5, min_count = 5 | max_iter = 4000 | 0.884 | 0.882 |
|vector_size = 200, sg = 1, window = 5, min_count = 5 | max_iter = 3000 | 0.897 | 0.8899 |
|vector_size = 200, sg = 1, window = 5, min_count = 5 | max_iter = 2000, C = 1.2 | 0.8974 | 0.89 |
|vector_size = 300, sg = 1, window = 5, min_count = 5 | max_iter = 4000, C = 0.9 | 0.998 | 0.8934 |
|vector_size = 1000, sg = 1, window = 5, min_count = 5 | max_iter = 4000, C = 0.9 | 0.9095 | 0.8947 |

In [133]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform

param_dist = {
    'C': uniform(0.01, 10),
    'solver': ['liblinear', 'saga', 'newton-cholesky', 'lbfgs'],
    'max_iter': [100, 500, 1000],
    'penalty': ['l1', 'l2']
}

random_search = RandomizedSearchCV(LogisticRegression(), param_distributions=param_dist, n_iter=100, cv=5)
random_search.fit(X_w2v_train, y_train)
print(random_search.best_params_)


d:\Anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

{'C': 1.4596835617148962, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'saga'}


d:\Anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
{'C': 1.4596835617148962, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'saga'}

### Random forest

### MLP

### Ensemble model

[0.5,1.5,0.75,0.75,1.5] --> 0.855 \
weights --> 0.8466

## Tổng kết

| Mô hình Word Embeddings| Dữ liệu |Mô hình ML| Tham số | training accuracy | testing accuracy | Đánh giá | 
|---------|------|----------|------|--------|----|---|
|Word2Vec|Không loại stop words |Decision tree | default | 0.979 | 0.7249 | Overfitting |
|Word2Vec|Không loại stop words |Logistic Regression | max_iter = 400 | 0.852 | 0.850 | -- |
|Word2Vec|Không loại stop words |Random forest | default | 0.99997 | 0.818 | Ovefitting |
|Word2Vec|Không loại stop words  |XGBoost | default | 0.961 | 0.833 | Ovefitting |
|Word2Vec|Không loại stop words |MLP | max_iter = 250, learning_rate_init = 0.0005 | 0.868 | 0.850 | -- |
|Word2Vec|Loại stop words |Decision tree | default | 0.976 | 0.751 | Overfitting |
|Word2Vec|Loại stop words |Logistic Regression | default | 0.850 | 0.848 | -- |
|Word2Vec|Loại stop words |Random forest | default | 0.999 | 0.830 | Overfitting |
|Word2Vec|Loại stop words  |XGBoost | default | 0.964 | 0.834 | Ovefitting |
|Word2Vec|Loại stop words |MLP | default | 0.938 | 0.816 | Overfitting |
